In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Define the URL for the Manipulation Train set
repo_base = "https://raw.githubusercontent.com/ArkadiusDS/MIPD/master/data"
url = f"{repo_base}/manipulation/train.csv"

print(f"Downloading data from: {url}")

try:
    # Load the dataset into a global variable
    df_mipd_manipulation = pd.read_csv(url)
    print(f"Successfully loaded {len(df_mipd_manipulation)} records into 'df_mipd_manipulation'.")

except Exception as e:
    print(f"Error loading dataset: {e}")

df_mipd_manipulation.info()

Successfully loaded 10749 records into 'df_mipd_manipulation'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10749 entries, 0 to 10748
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   article              10749 non-null  object
 1   REFERENCE_ERROR      10749 non-null  bool  
 2   WHATABOUTISM         10749 non-null  bool  
 3   STRAWMAN             10749 non-null  bool  
 4   EMOTIONAL_CONTENT    10749 non-null  bool  
 5   CHERRY_PICKING       10749 non-null  bool  
 6   FALSE_CAUSE          10749 non-null  bool  
 7   MISLEADING_CLICKBAI  10749 non-null  bool  
 8   ANECDOTE             10749 non-null  bool  
 9   LEADING_QUESTIONS    10749 non-null  bool  
 10  EXAGGERATION         10749 non-null  bool  
 11  QUOTE_MINING         10749 non-null  bool  
dtypes: bool(11), object(1)
memory usage: 199.6+ KB


In [ ]:
system_instruction='''
Jesteś ekspertem ds. weryfikacji informacji i analizy mediów. Twoim zadaniem jest przeanalizowanie podanego artykułu i zidentyfikowanie w nim technik manipulacji.

Dozwolone kategorie manipulacji to:
- REFERENCE_ERROR
- WHATABOUTISM
- STRAWMAN
- EMOTIONAL_CONTENT
- CHERRY_PICKING
- FALSE_CAUSE
- MISLEADING_CLICKBAIT
- ANECDOTE
- LEADING_QUESTIONS
- EXAGGERATION
- QUOTE_MINING

Instrukcja:
1. Przeanalizuj dokładnie treść podanego artykułu.
2. Dopasuj fragmenty tekstu do powyższych definicji.
3. Zwróć wynik WYŁĄCZNIE w formacie JSON z kluczem "discovered_techniques".
4. Jeśli nie wykryto żadnych technik, zwróć pustą listę.
5. Nie generuj żadnych dodatkowych wyjaśnień, wstępów ani komentarzy.

Przykład wyjścia:
{"discovered_techniques": ["EMOTIONAL_CONTENT", "EXAGGERATION"]}
'''

In [ ]:
import json

techniques_columns = df_mipd_manipulation.columns[df_mipd_manipulation.columns.get_loc('REFERENCE_ERROR'):df_mipd_manipulation.columns.get_loc('QUOTE_MINING')+1].tolist()
print(f"Defined manipulation techniques columns: {techniques_columns}")

def format_record(row):
    """Przetwarza jeden wiersz DataFrame na format treningowy."""

    # 1. Wyciągnij aktywne techniki (gdzie wartość w kolumnie jest True/1)
    active_techniques = [col for col in techniques_columns if row[col]]

    # 2. Stwórz Target (Output) w formacie JSON z Markdown
    # ensure_ascii=False jest KLUCZOWE dla polskich znaków
    json_content = json.dumps({"discovered_techniques": active_techniques}, ensure_ascii=False)
    output_str = f"```json\n{json_content}\n```"

    # 3. Zwróć słownik w formacie dla Unsloth
    return {
        "instruction": system_instruction.strip(),
        "input": row['article'],  # Zakładam, że kolumna z tekstem to 'article'
        "output": output_str
    }

# Aplikujemy funkcję do DataFrame (tworzy nowy DataFrame z wynikami)
# axis=1 przetwarza wiersz po wierszu
formatted_df = df_mipd_manipulation.apply(format_record, axis=1, result_type='expand')

# Zapis do pliku JSONL
# orient='records' + lines=True tworzy format JSONL
# force_ascii=False zachowuje polskie ogonki w pliku
output_file = "/content/drive/MyDrive/mipd_train.jsonl"
formatted_df.to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(formatted_df)} rekordów do pliku: {output_file}")
print("Przykładowy rekord:", formatted_df.iloc[0].to_dict())

Defined manipulation techniques columns: ['REFERENCE_ERROR', 'WHATABOUTISM', 'STRAWMAN', 'EMOTIONAL_CONTENT', 'CHERRY_PICKING', 'FALSE_CAUSE', 'MISLEADING_CLICKBAI', 'ANECDOTE', 'LEADING_QUESTIONS', 'EXAGGERATION', 'QUOTE_MINING']
Zapisano 10749 rekordów do pliku: mipd_train.jsonl
Przykładowy rekord: {'instruction': 'Jesteś ekspertem ds. weryfikacji informacji i analizy mediów. Twoim zadaniem jest przeanalizowanie podanego artykułu i zidentyfikowanie w nim technik manipulacji.\n\nDozwolone kategorie manipulacji to:\n- REFERENCE_ERROR\n- WHATABOUTISM\n- STRAWMAN\n- EMOTIONAL_CONTENT\n- CHERRY_PICKING\n- FALSE_CAUSE\n- MISLEADING_CLICKBAIT\n- ANECDOTE\n- LEADING_QUESTIONS\n- EXAGGERATION\n- QUOTE_MINING\n\nInstrukcja:\n1. Przeanalizuj dokładnie treść podanego artykułu.\n2. Dopasuj fragmenty tekstu do powyższych definicji.\n3. Zwróć wynik WYŁĄCZNIE w formacie JSON z kluczem "discovered_techniques".\n4. Jeśli nie wykryto żadnych technik, zwróć pustą listę.\n5. Nie generuj żadnych dodatkowy

In [ ]:
url = f"{repo_base}/manipulation/validation.csv"

print(f"Downloading data from: {url}")

try:
    # Load the dataset into a global variable
    df_mipd_manipulation_val = pd.read_csv(url)
    print(f"Successfully loaded {len(df_mipd_manipulation_val)} records into 'df_mipd_manipulation_val'.")

except Exception as e:
    print(f"Error loading dataset: {e}")

df_mipd_manipulation_val.info()

url = f"{repo_base}/manipulation/test.csv"

print(f"Downloading data from: {url}")

try:
    # Load the dataset into a global variable
    df_mipd_manipulation_test = pd.read_csv(url)
    print(f"Successfully loaded {len(df_mipd_manipulation_test)} records into 'df_mipd_manipulation_test'.")

except Exception as e:
    print(f"Error loading dataset: {e}")

df_mipd_manipulation_test.info()

Successfully loaded 3086 records into 'df_mipd_manipulation_val'.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3086 entries, 0 to 3085
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   article              3086 non-null   object
 1   REFERENCE_ERROR      3086 non-null   bool  
 2   WHATABOUTISM         3086 non-null   bool  
 3   STRAWMAN             3086 non-null   bool  
 4   EMOTIONAL_CONTENT    3086 non-null   bool  
 5   CHERRY_PICKING       3086 non-null   bool  
 6   FALSE_CAUSE          3086 non-null   bool  
 7   MISLEADING_CLICKBAI  3086 non-null   bool  
 8   ANECDOTE             3086 non-null   bool  
 9   LEADING_QUESTIONS    3086 non-null   bool  
 10  EXAGGERATION         3086 non-null   bool  
 11  QUOTE_MINING         3086 non-null   bool  
dtypes: bool(11), object(1)
memory usage: 57.4+ KB
Successfully loaded 1521 records into 'df_mipd_manipulation_test'.
<class 'pandas.core.fra

In [ ]:
# Aplikujemy funkcję do DataFrame (tworzy nowy DataFrame z wynikami)
# axis=1 przetwarza wiersz po wierszu
formatted_df_val = df_mipd_manipulation_val.apply(format_record, axis=1, result_type='expand')

# Zapis do pliku JSONL
# orient='records' + lines=True tworzy format JSONL
# force_ascii=False zachowuje polskie ogonki w pliku
output_file = "/content/drive/MyDrive/mipd_val.jsonl"
formatted_df_val.to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(formatted_df_val)} rekordów do pliku: {output_file}")
print("Przykładowy rekord:", formatted_df_val.iloc[0].to_dict())


# Aplikujemy funkcję do DataFrame (tworzy nowy DataFrame z wynikami)
# axis=1 przetwarza wiersz po wierszu
formatted_df_test = df_mipd_manipulation_test.apply(format_record, axis=1, result_type='expand')

# Zapis do pliku JSONL
# orient='records' + lines=True tworzy format JSONL
# force_ascii=False zachowuje polskie ogonki w pliku
output_file = "/content/drive/MyDrive/mipd_test.jsonl"
formatted_df_test.to_json(output_file, orient='records', lines=True, force_ascii=False)

print(f"Zapisano {len(formatted_df_test)} rekordów do pliku: {output_file}")
print("Przykładowy rekord:", formatted_df_test.iloc[0].to_dict())

Zapisano 3086 rekordów do pliku: mipd_val.jsonl
Przykładowy rekord: {'instruction': 'Jesteś ekspertem ds. weryfikacji informacji i analizy mediów. Twoim zadaniem jest przeanalizowanie podanego artykułu i zidentyfikowanie w nim technik manipulacji.\n\nDozwolone kategorie manipulacji to:\n- REFERENCE_ERROR\n- WHATABOUTISM\n- STRAWMAN\n- EMOTIONAL_CONTENT\n- CHERRY_PICKING\n- FALSE_CAUSE\n- MISLEADING_CLICKBAIT\n- ANECDOTE\n- LEADING_QUESTIONS\n- EXAGGERATION\n- QUOTE_MINING\n\nInstrukcja:\n1. Przeanalizuj dokładnie treść podanego artykułu.\n2. Dopasuj fragmenty tekstu do powyższych definicji.\n3. Zwróć wynik WYŁĄCZNIE w formacie JSON z kluczem "discovered_techniques".\n4. Jeśli nie wykryto żadnych technik, zwróć pustą listę.\n5. Nie generuj żadnych dodatkowych wyjaśnień, wstępów ani komentarzy.\n\nPrzykład wyjścia:\n{"discovered_techniques": ["EMOTIONAL_CONTENT", "EXAGGERATION"]}', 'input': '"Szczepienia przeciw COVID-19: pora dnia moŋe mieæ znaczenie" "Więcej podobnych tematów na stroni